In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import lxml.html as lh
import re
import json



In [2]:
states= pd.read_csv('../csv/states.csv')
states=states.rename(columns={"State.1": "StateAbv"}).reset_index(drop=True)
# Just to be sure no spaces remain in column for merge!!! 
states.StateAbv = states.StateAbv.str.strip()
# states.convert_dtypes()
states.head()

,State,StateAbv,State Capital,Region
0,Alabama,AL,Montgomery,South
1,Alaska,AK,Juneau,West
2,Arizona,AZ,Phoenix,West
3,Arkansas,AR,Little Rock,South
4,California,CA,Sacramento,West


In [3]:
state =states["StateAbv"]
nat_park=[]  #This is where it all winds up

for name in state:
    st_park = {} # will be a dict of lists of all st parks- resets on each loop   
    
    url=f'https://stateparks.com/{name}.html'
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    results= soup.find('div', id="overview")
    body = results.find_all('div', style="display:inline-block;")
    
    for row_num in range(len(body)): 
        park = [] # this will hold entries for one park- resets each loop
        for row_item in body[row_num]:
            aa = re.sub("(\xa0)|(\n)|,|•| |''"," ",row_item.text)
            #remove opening blank before some entries
            aa= aa.lstrip(' ')
        
            #assign value to the parks where there is just one, scraping give an empty value
            if aa == '':
                aa = '1'
            #beacuse of web formatting, some parks had empty rows at the front
            if len(park) >1:
                del park[0]  
            park.append(aa)
            # from here park is a 2 item list
            
        def Convert(a):#This will create a dictionary per park with its count 
            #First create an iterator, and initialize it to variable ‘it’
            it = iter(a)
            #Then use zip method, to zip keys and values together. 
            #Finally typecast it to dict type. 
            res_dct = dict(zip(it, it))
            #flip the key value pair so type comes before count
            res_dct2= {v: k for k,v in res_dct.items()}
            #then take that and convert all values to int
            res_dct3 ={str(k):int(v) for k,v in res_dct2.items()}
            return res_dct3
        
        #send park to the Convert function 
        park_dict = Convert(park)
        #add park to state park list     
        st_park.update(park_dict)
  # add state park list of individual park dicts to nationl parks list which becomes our df 
    nat_park.append(st_park)

In [4]:
st_park

{'National Memorial': 1, 'National Historic Sites': 2}

In [5]:
df=pd.DataFrame(nat_park)

df.dtypes

State Parks                        float64
State Forests                      float64
State Wildlife Management Areas    float64
National Park                      float64
National Forests                   float64
                                    ...   
State Nursery                      float64
State Preserve                     float64
State Scenic Rivers                float64
State Fishery Areas                float64
Trail                              float64
Length: 61, dtype: object

In [6]:
df = df.fillna(0)

In [7]:
# df = df.apply(pd.to_numeric, downcast='integer')
df = df.astype(int, errors='ignore')

In [8]:
df
df["AllParks"]=  df.sum(axis=1)

In [9]:
df['park_rank'] = df['AllParks'].rank(method='max', na_option='bottom').astype(int)
df

,State Parks,State Forests,State Wildlife Management Areas,National Park,National Forests,National Historic Sites,National Historic Park,National Monuments,National Wildlife Refuges,National Parks,...,National Seashores,Army Corps of Engineers,State Nature Preserves,State Nursery,State Preserve,State Scenic Rivers,State Fishery Areas,Trail,AllParks,park_rank
0,24,3,4,1,4,2,1,2,10,0,...,0,0,0,0,0,0,0,0,51,19
1,4,0,0,0,2,0,0,2,22,2,...,0,0,0,0,0,0,0,0,42,12
2,16,0,0,0,8,0,0,9,7,4,...,0,0,0,0,0,0,0,0,57,24
3,36,0,12,1,3,3,0,0,9,0,...,0,0,0,0,0,0,0,0,71,33
4,109,6,0,0,19,3,0,5,31,9,...,0,0,0,0,0,0,0,0,235,51
5,6,2,0,0,11,0,0,0,4,3,...,0,0,0,0,0,0,0,0,36,11
6,53,25,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,81,38
7,15,2,3,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,23,5
8,114,5,0,0,3,0,0,0,20,3,...,0,0,0,0,0,0,0,0,190,50
9,53,0,0,0,2,0,0,0,9,0,...,0,0,0,0,0,0,0,0,89,41


In [10]:
#join statedata to park data 
park_dicts= df.join(states)
#reorder columns to make state easier to find in json
park_dicts = park_dicts[['State',
       'StateAbv', 'State Capital', 'Region','AllParks','park_rank','State Parks', 'State Forests', 'State Wildlife Management Areas',
       'National Park', 'National Forests', 'National Historic Sites',
       'National Historic Park', 'National Monuments',
       'National Wildlife Refuges', 'National Parks',
       'National Historic Parks', 'National Recreation Trail',
       'National Park and Preserves', 'State Recreation Area',
       'State Historic Site', 'State Fish Hatchery', 'State Wildlife Area',
       'National Memorial', 'National Historic Site', 'National Wildlife Area',
       'National Recreation Area', 'National Recreation Trails',
       'Metro and City Parks', 'State Forest', 'State Fish Hatcherys',
       'State Natural Area', 'State Recreation Areas',
       'State Historic Sites', 'State Nature Preserve', 'State Reserves',
       'State Wildlife Areas', 'National Recreation Areas',
       'National Seashore', 'National Park and Preserve',
       'National Historic Landmark', 'National Monument', 'State Memorial',
       'State Preserves', 'National Memorials',
       'State Wildlife Management Area', 'State Wildlife Refuge',
       'Army Corps of Engineerss', 'Metro and City Parkss',
       'State Memorials', 'State Natural Areas', 'National Forest',
       'State Fishing Area', 'National Wildlife Refuge', 'State Reserve',
       'State Airport', 'National Historic Landmarks',
       'National Wilderness Areas', 'National Natural Area',
       'National Seashores', 'Army Corps of Engineers',
       'State Nature Preserves', 'State Nursery', 'State Preserve',
       'State Scenic Rivers', 'State Fishery Areas', 'Trail']]
park_dicts

,State,StateAbv,State Capital,Region,AllParks,park_rank,State Parks,State Forests,State Wildlife Management Areas,National Park,...,National Wilderness Areas,National Natural Area,National Seashores,Army Corps of Engineers,State Nature Preserves,State Nursery,State Preserve,State Scenic Rivers,State Fishery Areas,Trail
0,Alabama,AL,Montgomery,South,51,19,24,3,4,1,...,0,0,0,0,0,0,0,0,0,0
1,Alaska,AK,Juneau,West,42,12,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arizona,AZ,Phoenix,West,57,24,16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arkansas,AR,Little Rock,South,71,33,36,0,12,1,...,0,0,0,0,0,0,0,0,0,0
4,California,CA,Sacramento,West,235,51,109,6,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Colorado,CO,Denver,West,36,11,6,2,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Connecticut,CT,Hartford,Northeast,81,38,53,25,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Delaware,DE,Dover,Northeast,23,5,15,2,3,0,...,0,0,0,0,0,0,0,0,0,0
8,Florida,FL,Tallahassee,South,190,50,114,5,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Georgia,GA,Atlanta,South,89,41,53,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#have to sort the df by state name or the loop in json append does not work
park_dict=park_dicts.sort_values('State')

park_dict

,State,StateAbv,State Capital,Region,AllParks,park_rank,State Parks,State Forests,State Wildlife Management Areas,National Park,...,National Wilderness Areas,National Natural Area,National Seashores,Army Corps of Engineers,State Nature Preserves,State Nursery,State Preserve,State Scenic Rivers,State Fishery Areas,Trail
0,Alabama,AL,Montgomery,South,51,19,24,3,4,1,...,0,0,0,0,0,0,0,0,0,0
1,Alaska,AK,Juneau,West,42,12,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arizona,AZ,Phoenix,West,57,24,16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arkansas,AR,Little Rock,South,71,33,36,0,12,1,...,0,0,0,0,0,0,0,0,0,0
4,California,CA,Sacramento,West,235,51,109,6,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Colorado,CO,Denver,West,36,11,6,2,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Connecticut,CT,Hartford,Northeast,81,38,53,25,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Delaware,DE,Dover,Northeast,23,5,15,2,3,0,...,0,0,0,0,0,0,0,0,0,0
50,District of Columbia,DC,Washington,Northeast,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Florida,FL,Tallahassee,South,190,50,114,5,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
park_dict.to_csv("../csv/all_parks.csv", index=False)

In [13]:
park_dict.to_json(r'../json/all_parks.json',orient="records")